# tem Beneficios a entrega contínua?

## Revisar literatura para encontrar melhores perguntas de pesquisa

## Analisar dataset para estabelecer quais projetos tem entrega contínua



In [87]:
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId
client = MongoClient()
db = client.smartshark_2_2

### 1) Tem integração contínua - OK

In [88]:
vcs_systems = pd.DataFrame(list(db.vcs_system.find()))
travisBuilds = pd.DataFrame(list(db.travis_build.find()))
projetosComBuilds = travisBuilds['vcs_system_id'].unique()
vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()

,_id,url,project_id,last_updated,repository_type,repository_file
0,5bef24dc35e3ea2b7bc08208,https://github.com/apache/commons-rdf.git,5beebd870da3ef21a62b756e,2018-11-16 21:13:16.874,git,5dcbf72e2ecf182bf5d3ab89
2,58bfd79dfa94f2c7f0d32551,https://github.com/apache/commons-math,58bfd5860da3ef7cef464c78,2018-02-15 11:35:53.876,git,5dcbf5ce2ecf182bf5d37773
3,5b1fa04730a71b06bc3734c1,https://github.com/apache/mahout,5b1f9fbb0da3ef49b7d0a480,2018-06-12 12:28:23.953,git,5dcbf61c2ecf182bf5d382c4
4,5b27c64830a71b06bceb0ead,https://github.com/apache/opennlp.git,5b27c4cc0da3ef789daec0a1,2018-06-18 16:48:40.415,git,5dcbf6262ecf182bf5d3852f
5,5b34a9c830a71b06bc3d9ef3,https://github.com/apache/calcite.git,5b34a9210da3ef789cc240ec,2018-06-28 11:26:32.261,git,5dcbf6362ecf182bf5d38708
6,5b3b305530a71b06bc5bba5b,https://github.com/apache/cayenne.git,5b3b2d520da3ef789ec953ea,2018-07-03 10:14:13.878,git,5dcbf63c2ecf182bf5d387d8
8,5da726db3dfe5ddb85120d96,https://github.com/apache/bigtop.git,5da724f20da3ef03c84bcf48,2019-10-16 16:19:07.019,git,5dcbf7d72ecf182bf5d3c55d
9,5c87a755504acf99a4dbf833,https://github.com/apache/commons-bcel.git,5c87a7300da3ef6c8a4bf6d8,2019-03-12 13:34:29.466,git,5dcbf7602ecf182bf5d3b19e
10,5bed279735e3ea2b7b9d059e,https://github.com/apache/commons-beanutils.git,5bed276c0da3ef21a58b337a,2018-11-15 09:00:23.400,git,5dcbf7172ecf182bf5d3a90b
11,5bed5a4635e3ea2b7ba10f97,https://github.com/apache/commons-codec.git,5bed58010da3ef21a58b337b,2018-11-15 12:36:38.541,git,5dcbf7182ecf182bf5d3a91d


### 2) Pegar as tags do projetos de integração contínua? - OK

In [89]:
tags = pd.DataFrame(list(db.tag.find()))
idsVcsComBuilds = vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()._id

In [90]:
tagsDosProjComBuild = tags.query('vcs_system_id.isin(@idsVcsComBuilds)').drop_duplicates().dropna()

In [91]:
tagsDosProjComBuild

,_id,name,commit_id,vcs_system_id,message,tagger_id,date,date_offset
3,5bef24e783ad2f56e01ca88c,0.3.0-incubating,5bef24e783ad2f56e01ca88a,5bef24dc35e3ea2b7bc08208,Apache Commons RDF 0.3.0-incubating\n\nhttps:/...,58c1133102ca40f8bfb1e2a9,2016-11-15 13:28:17,0.0
5,5bef254783ad2f56d01caefd,0.1.0-incubating,5bef254683ad2f56d01caefb,5bef24dc35e3ea2b7bc08208,[maven-release-plugin] copy for tag 0.1.0-incu...,58bfd1cb02ca40f8bf1485a4,2015-05-07 14:10:41,120.0
27,58bfd7a53cff546cd3b42b9e,MATH_3_4_1_RC1,58bfd7a53cff546cd3b42b9a,58bfd79dfa94f2c7f0d32551,Creating Apache Commons Math v3.4.1 RC1 tag.\n...,58bfd79d3cff546cc7b42a28,2015-01-08 16:51:54,60.0
28,58bfd7a53cff546cd8b42c1b,MATH_3_4_RC2,58bfd7a53cff546cd8b42c17,58bfd79dfa94f2c7f0d32551,Creating Apache Commons Math v3.4 RC2 tag.\n--...,58bfd79d3cff546cc7b42a28,2014-12-19 14:41:18,60.0
29,58bfd7b83cff546cc8b43dab,MATH_1_2,58bfd7b83cff546cc8b43da7,58bfd79dfa94f2c7f0d32551,MATH_1_2\n,58bfd7a53cff546ccbb42b96,2008-02-24 05:56:08,0.0
...,...,...,...,...,...,...,...,...
4780,615e91666511689f63ec2c87,qpid-jms-0.1.0-beta,615e91666511689f63ec2c85,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag qpid-jms-0...,5a031143efa7b46cc724c03b,2015-03-09 18:24:25,0.0
4781,615e91ae6511689f63ec816c,1.2.0,615e91ae6511689f63ec816a,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.2.0\n,5e71cec61c4f2a5a2b6418d7,2021-09-24 12:36:32,60.0
4782,615e91af6511689f63ec8221,1.1.0,615e91af6511689f63ec821f,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.1.0\n,5f27cd89af02e2d6de7c7cdc,2021-07-05 15:35:44,60.0
4783,615e91b06511689f63ec8294,1.0.0,615e91b06511689f63ec8292,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.0.0\n,5f27cd89af02e2d6de7c7cdc,2021-05-19 14:58:57,60.0


### 3) Determinei oq vou considerar Tag de Release
##### Tags de release não tem no nome os termos 'RC', 'candidate', 'test', 'beta', 'alfa', 'unreleased', snapshot, siteupdate, freeze, sandbox, migration

In [92]:
tagsDeRelase = tagsDosProjComBuild[
    tagsDosProjComBuild['name']
    .str.contains('rc|RC|candidate|CANDIDATE|test|TEST|beta|BETA|alpha|ALPHA|unreleased|UNRELEASED|snapshot|SNAPSHOT|siteupdate|SITEUPDATE|freeze|FREEZE|sandbox|SANDBOX|migration|MIGRATION')==False].drop_duplicates().dropna()

In [93]:
tagsDeRelase.name.to_csv('tags.csv', index=False)

In [94]:
tagsDeRelase

,_id,name,commit_id,vcs_system_id,message,tagger_id,date,date_offset
3,5bef24e783ad2f56e01ca88c,0.3.0-incubating,5bef24e783ad2f56e01ca88a,5bef24dc35e3ea2b7bc08208,Apache Commons RDF 0.3.0-incubating\n\nhttps:/...,58c1133102ca40f8bfb1e2a9,2016-11-15 13:28:17,0.0
5,5bef254783ad2f56d01caefd,0.1.0-incubating,5bef254683ad2f56d01caefb,5bef24dc35e3ea2b7bc08208,[maven-release-plugin] copy for tag 0.1.0-incu...,58bfd1cb02ca40f8bf1485a4,2015-05-07 14:10:41,120.0
29,58bfd7b83cff546cc8b43dab,MATH_1_2,58bfd7b83cff546cc8b43da7,58bfd79dfa94f2c7f0d32551,MATH_1_2\n,58bfd7a53cff546ccbb42b96,2008-02-24 05:56:08,0.0
32,58bfd7c33cff546cd1b447ff,MATH_3_1,58bfd7c33cff546cd1b447fb,58bfd79dfa94f2c7f0d32551,MATH_3_1\n,58bfd7a53cff546ccab42b89,2012-12-23 18:09:22,0.0
33,58bfd7cd3cff546cd5b44d22,MATH_3_5,58bfd7cd3cff546cd5b44d1c,58bfd79dfa94f2c7f0d32551,RC2 becomes the 3.5 official version.\n-----BE...,58bfd79d3cff546cc7b42a28,2015-04-17 14:39:07,120.0
...,...,...,...,...,...,...,...,...
4779,615e91646511689f63ec2b38,qpid-jms-0.1.0,615e91646511689f63ec2b36,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag qpid-jms-0...,5a031143efa7b46cc724c03b,2015-03-16 18:45:53,0.0
4781,615e91ae6511689f63ec816c,1.2.0,615e91ae6511689f63ec816a,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.2.0\n,5e71cec61c4f2a5a2b6418d7,2021-09-24 12:36:32,60.0
4782,615e91af6511689f63ec8221,1.1.0,615e91af6511689f63ec821f,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.1.0\n,5f27cd89af02e2d6de7c7cdc,2021-07-05 15:35:44,60.0
4783,615e91b06511689f63ec8294,1.0.0,615e91b06511689f63ec8292,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.0.0\n,5f27cd89af02e2d6de7c7cdc,2021-05-19 14:58:57,60.0


### 4) Filtrar builds que tem commits de tag de release

In [132]:
commitsDeRelease = tagsDeRelase.commit_id.unique()
buildsIntegrados = travisBuilds.query('commit_id.isin(@commitsDeRelease)')
stateIndesejados = ['failed', 'canceled']
BuildsValidos = buildsIntegrados.query('~state.isin(@stateIndesejados)')

In [148]:
BuildsValidos.state.value_counts()

passed     216
errored     48
Name: state, dtype: int64

### 5) Analisar quais projetos tem builds de entrega (34 no total)

In [150]:
vcsComEntrega = BuildsValidos.vcs_system_id.unique()
vcs_systems.query('_id.isin(@vcsComEntrega)').drop_duplicates().dropna()

,_id,url,project_id,last_updated,repository_type,repository_file
0,5bef24dc35e3ea2b7bc08208,https://github.com/apache/commons-rdf.git,5beebd870da3ef21a62b756e,2018-11-16 21:13:16.874,git,5dcbf72e2ecf182bf5d3ab89
4,5b27c64830a71b06bceb0ead,https://github.com/apache/opennlp.git,5b27c4cc0da3ef789daec0a1,2018-06-18 16:48:40.415,git,5dcbf6262ecf182bf5d3852f
5,5b34a9c830a71b06bc3d9ef3,https://github.com/apache/calcite.git,5b34a9210da3ef789cc240ec,2018-06-28 11:26:32.261,git,5dcbf6362ecf182bf5d38708
6,5b3b305530a71b06bc5bba5b,https://github.com/apache/cayenne.git,5b3b2d520da3ef789ec953ea,2018-07-03 10:14:13.878,git,5dcbf63c2ecf182bf5d387d8
8,5da726db3dfe5ddb85120d96,https://github.com/apache/bigtop.git,5da724f20da3ef03c84bcf48,2019-10-16 16:19:07.019,git,5dcbf7d72ecf182bf5d3c55d
9,5c87a755504acf99a4dbf833,https://github.com/apache/commons-bcel.git,5c87a7300da3ef6c8a4bf6d8,2019-03-12 13:34:29.466,git,5dcbf7602ecf182bf5d3b19e
11,5bed5a4635e3ea2b7ba10f97,https://github.com/apache/commons-codec.git,5bed58010da3ef21a58b337b,2018-11-15 12:36:38.541,git,5dcbf7182ecf182bf5d3a91d
12,5bed25af35e3ea2b7b8bb711,https://github.com/apache/commons-collections.git,5bed254e0da3ef21a62b756d,2018-11-15 08:52:15.394,git,5dcbf7162ecf182bf5d3a8bb
13,5bed61cc35e3ea2b7b2a1805,https://github.com/apache/commons-compress.git,5bed583a0da3ef21a58b337c,2018-11-15 13:08:44.599,git,5dcbf71b2ecf182bf5d3a931
14,5bed83ca35e3ea2b7be8d7c9,https://github.com/apache/commons-configuratio...,5bed83440da3ef21a2107882,2018-11-15 15:33:46.623,git,5dcbf71d2ecf182bf5d3a999


### 6) Desses projetos quais tem uma quantidade relevante de entregas que permita a nossa análise (>4 Builds) (12 projetos no total)

In [135]:
#Refazer
BuildsValidosPorCommit = BuildsValidos.groupby(by='commit_id').max()

In [136]:
BuildsValidosPorCommit

,_id,tr_id,vcs_system_id,number,state,duration,event_type,started_at,finished_at,stages,pr_number
commit_id,,,,,,,,,,,
5b23df3ca1501a253fd0c9ac,5f4506ed1f9c0442163994f9,348202462,5b23df3c30a71b06bc01e16a,783,passed,2266.0,push,2018-03-02 12:28:33,2018-03-02 12:52:18,[],NaN
5b23df3da1501a2544d0c9e7,5f45062f1f9c044216399493,339357997,5b23df3c30a71b06bc01e16a,748,passed,1190.0,push,2018-02-09 08:46:10,2018-02-09 09:06:05,[],NaN
5b23df51a1501a2540d0ccbf,5f44fd351f9c044216398f63,195812790,5b23df3c30a71b06bc01e16a,291,passed,475.0,push,2017-01-27 09:35:44,2017-01-27 09:40:36,[],NaN
5b23df59a1501a2544d0d140,5f44fcbf1f9c044216398f18,180629675,5b23df3c30a71b06bc01e16a,266,passed,468.0,push,2016-12-02 14:17:41,2016-12-02 14:22:25,[],NaN
5b23df61a1501a2544d0d306,5f44fbeb1f9c044216398e73,165145794,5b23df3c30a71b06bc01e16a,210,passed,503.0,push,2016-10-05 05:45:25,2016-10-05 05:50:19,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...
614da93b593343690afd26e8,615c0de6d338a9992c3eb161,513437340,614da8f1f73e2aa390da0ece,53,errored,2399.0,push,2019-03-30 12:32:53,2019-03-30 13:12:52,[],NaN
615e91096511689f63ebb9cd,6179532ffdcba5d070a7a151,744924676,615e9105f73e2aa390c8a69b,1054,passed,849.0,push,2020-11-20 16:20:28,2020-11-20 16:26:21,[],NaN
615e910a6511689f63ebba86,617952e7fdcba5d070a7a121,717554528,615e9105f73e2aa390c8a69b,1042,passed,823.0,push,2020-08-13 13:09:27,2020-08-13 13:16:29,[],NaN


In [154]:
BuildsValidos.vcs_system_id.value_counts()

,vcs_system_id
5bd0268235e3ea2b7bbfdbae,32
5d8ccfb33dfe5ddb8590d841,28
5b23df3c30a71b06bc01e16a,22
5b43737430a71b06bcbdd168,21
5cd2806d504acf99a4cd728c,17
5c87a96c504acf99a4de5a18,14
5bbe0f4057674ee16793f245,13
5bed61cc35e3ea2b7b2a1805,12
5b27c64830a71b06bceb0ead,12
5b7ffe9530a71b06bc70c038,9


### 11 projetos foram selecionados 20 projetos e validado manualmemente se existia entrega contínua implementada de maneira coeerente. Ficaram 11
NIFI

STRUTS

kylin

httpcomponents-core (+/- datas muito espassadas)

commons-dbcp

commons-lang

httpcomponents-client

commons-compress

opennlp

curator

cayenne

manifoldcf

In [215]:
a = ObjectId('5bd0268235e3ea2b7bbfdbae')
tagsDeRelase.query('vcs_system_id == @a').sort_values(by=['date'])

,_id,name,commit_id,vcs_system_id,message,tagger_id,date,date_offset
2297,5bd02731bad86c1df2dd2b07,nifi-0.4.0,5bd02731bad86c1df2dd2b05,5bd0268235e3ea2b7bbfdbae,NIFI-1122 signed release tag for approved rele...,5bd02684bad86c1deddd00ac,2015-12-12 01:11:07,-300.0
2296,5bd02730bad86c1deadd2ae6,nifi-0.4.1,5bd02730bad86c1deadd2ae2,5bd0268235e3ea2b7bbfdbae,NIFI-1312 Signed release tag for approved rele...,5bd02684bad86c1deddd00ac,2015-12-22 15:48:07,-300.0
2294,5bd02722bad86c1df2dd2833,nifi-0.5.0,5bd02721bad86c1df2dd282f,5bd0268235e3ea2b7bbfdbae,NIFI-1379: Tag 0.5.0\n-----BEGIN PGP SIGNATURE...,5bc58bb657674ee167e32edc,2016-02-17 02:41:15,-300.0
2326,5bd028dabad86c1de5dd7307,nifi-0.5.1,5bd028dabad86c1de5dd7305,5bd0268235e3ea2b7bbfdbae,NIFI-1556: Signed release tag for approved rel...,5bc58bb657674ee167e32edc,2016-02-27 03:35:30,-300.0
2292,5bd02705bad86c1deadd253b,nifi-0.6.0,5bd02705bad86c1deadd2537,5bd0268235e3ea2b7bbfdbae,NIFI-1634 Signed release tag for approved rele...,5bd02683bad86c1de7dd00a4,2016-03-27 04:57:43,-240.0
2339,5bd0290abad86c1de0dd8315,nifi-0.6.1,5bd02909bad86c1de0dd8313,5bd0268235e3ea2b7bbfdbae,NIFI-1729 Signed Release Tag of nifi-0.6.1\n--...,5bd02684bad86c1deddd00ac,2016-04-16 19:28:22,-240.0
2312,5bd02869bad86c1decdd6b28,nifi-0.7.0,5bd02869bad86c1decdd6b26,5bd0268235e3ea2b7bbfdbae,NIFI-2112 Signed release tag for approved rele...,5bd026debad86c1deedd1592,2016-07-12 22:50:16,-240.0
2289,5bd026e5bad86c1de7dd18f1,nifi-1.0.0,5bd026e5bad86c1de7dd18ef,5bd0268235e3ea2b7bbfdbae,NIFI-2676 Signed release tag for approved rele...,5bd026debad86c1deedd1592,2016-08-31 05:45:12,600.0
2311,5bd02862bad86c1de7dd5559,nifi-0.7.1,5bd02861bad86c1de7dd5555,5bd0268235e3ea2b7bbfdbae,NIFI-2896 Signed release tag for approved rele...,5bd026c3bad86c1df0dd0d1b,2016-10-20 04:18:36,-240.0
2287,5bd026debad86c1df1dd14e1,nifi-1.1.0,5bd026debad86c1df1dd14df,5bd0268235e3ea2b7bbfdbae,NIFI-3100 signed release tag for approved rele...,5bd02684bad86c1deddd00ac,2016-11-29 12:52:02,-300.0


In [216]:
BuildsValidos.query('vcs_system_id == @a').sort_values(by=['started_at'])

,_id,tr_id,vcs_system_id,commit_id,number,state,duration,event_type,started_at,finished_at,stages,pr_number
62535,5fec528d5b4fe9ad4ae60aa0,150766238,5bd0268235e3ea2b7bbfdbae,5bd02869bad86c1decdd6b26,3044,passed,6418.0,push,2016-08-08 22:55:43,2016-08-08 23:32:14,[],NaN
63007,5fec58535b4fe9ad4ae60e86,155387591,5bd0268235e3ea2b7bbfdbae,5bd026e5bad86c1de7dd18ef,3516,passed,2333.0,push,2016-08-26 17:12:04,2016-08-26 17:50:57,[],NaN
63031,5fec58955b4fe9ad4ae60eb6,156115988,5bd0268235e3ea2b7bbfdbae,5bd026e5bad86c1de7dd18ef,3540,passed,2343.0,push,2016-08-30 03:48:13,2016-08-30 04:27:16,[],NaN
63577,5fec6364b69ad29625d1f2d9,168167153,5bd0268235e3ea2b7bbfdbae,5bd02861bad86c1de7dd5555,4086,passed,5778.0,push,2016-10-17 03:08:29,2016-10-17 03:41:23,[],NaN
63610,5fec63b7b69ad29625d1f31f,169128623,5bd0268235e3ea2b7bbfdbae,5bd02861bad86c1de7dd5555,4119,passed,5696.0,push,2016-10-20 04:22:44,2016-10-20 04:55:34,[],NaN
64005,5fec66b0b69ad29625d1f649,178999919,5bd0268235e3ea2b7bbfdbae,5bd026debad86c1df1dd14df,4514,passed,2202.0,push,2016-11-26 06:11:08,2016-11-26 06:47:50,[],NaN
64009,5fec66b6b69ad29625d1f651,179750939,5bd0268235e3ea2b7bbfdbae,5bd026debad86c1df1dd14df,4518,passed,2195.0,push,2016-11-29 13:11:30,2016-11-29 13:48:05,[],NaN
64180,5fec67cbb69ad29625d1f7a7,184366525,5bd0268235e3ea2b7bbfdbae,5bd02897bad86c1de5dd611b,4689,passed,2381.0,push,2016-12-15 21:41:29,2016-12-15 22:21:10,[],NaN
64202,5fec67ffb69ad29625d1f7d3,185290527,5bd0268235e3ea2b7bbfdbae,5bd028e2bad86c1de5dd75b1,4711,passed,2183.0,push,2016-12-19 21:58:42,2016-12-19 22:35:05,[],NaN
64241,5fec685fb69ad29625d1f821,186219635,5bd0268235e3ea2b7bbfdbae,5bd028e2bad86c1de5dd75b1,4750,passed,2181.0,push,2016-12-22 23:23:16,2016-12-22 23:59:37,[],NaN


Seleciondo os 13 projetos criar uma tabela com vsc id, project_id e issue_system_id e data de corte (data da tag de entrega)

In [217]:
BuildsValidosPorCommit.event_type.value_counts()

push    144
Name: event_type, dtype: int64